# Washington DC Biking data | Hourly Bike Count Prediction

# 3. Modeling for WorkingDays
Rafael Hernandez

In [18]:
# To automatically reload the function file 
%load_ext autoreload
%aimport My_Functions
%run My_Functions.py
%autoreload 1
%matplotlib inline
warnings.simplefilter(action='ignore')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
# To automatically reload the function file 
%load_ext autoreload
%aimport My_Functions
%run My_Functions.py
%autoreload 1

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
from dask.distributed import Client

client = Client()
client

Client Scheduler: tcp://127.0.0.1:56140 Dashboard: http://127.0.0.1:56141/status,Cluster Workers: 4 Cores: 4 Memory: 8.59 GB


In [8]:
# Load datasets
wd_h = dd.read_csv('./workingdays_data_prepared.csv')
wd_h['dteday']=dd.to_datetime(wd_h.dteday ,unit='ns')

In [9]:
wd_h.compute().head()

,instant,dteday,yr,holiday,workingday,temp,atemp,hum,windspeed,casual,...,hr_16,hr_17,hr_18,hr_19,hr_20,hr_21,hr_22,hr_23,hr_2,hr_3
0,48,2011-01-03,0,0,1,0.22,0.1970,0.44,0.3582,0,...,0,0,0,0,0,0,0,0,0,0
1,49,2011-01-03,0,0,1,0.20,0.1667,0.44,0.4179,0,...,0,0,0,0,0,0,0,0,0,0
2,50,2011-01-03,0,0,1,0.16,0.1364,0.47,0.3881,0,...,0,0,0,0,0,0,0,0,0,0
3,51,2011-01-03,0,0,1,0.16,0.1364,0.47,0.2836,0,...,0,0,0,0,0,0,0,0,0,0
4,52,2011-01-03,0,0,1,0.14,0.1061,0.50,0.3881,0,...,0,0,0,0,0,0,0,0,0,0


In [10]:
wd_h.compute().info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11865 entries, 0 to 11864
Data columns (total 73 columns):
instant                  11865 non-null int64
dteday                   11865 non-null datetime64[ns]
yr                       11865 non-null int64
holiday                  11865 non-null int64
workingday               11865 non-null int64
temp                     11865 non-null float64
atemp                    11865 non-null float64
hum                      11865 non-null float64
windspeed                11865 non-null float64
casual                   11865 non-null int64
registered               11865 non-null int64
cnt                      11865 non-null int64
isDaylight               11865 non-null bool
isNoon                   11865 non-null bool
relative_temp            11865 non-null float64
relative_atemp           11865 non-null float64
relative_hum             11865 non-null float64
relative_windspeed       11865 non-null float64
Wday_RushHour_High       11865 non-null 

## Subset Data

Use "train_test_split" for dask to subset the train vs test data failed, therefore the . dataset is split manually using the "dteday" from Q1/2011 to Q2/2012 - Train & Q3-Q4/2012 - test.

In [11]:
#X_train, X_test, y_train, y_test = train_test_split(X, y) #Note that does not workl for train_test_split from dask-ml

In [12]:
X_train = wd_h[wd_h['dteday'] < datetime.datetime(2012, 7, 5, 0, 0)].drop(['cnt','casual','registered','dteday','instant'],axis=1)
X_train.compute().head()

,yr,holiday,workingday,temp,atemp,hum,windspeed,isDaylight,isNoon,relative_temp,...,hr_16,hr_17,hr_18,hr_19,hr_20,hr_21,hr_22,hr_23,hr_2,hr_3
0,0,0,1,0.22,0.1970,0.44,0.3582,False,False,-1.611902,...,0,0,0,0,0,0,0,0,0,0
1,0,0,1,0.20,0.1667,0.44,0.4179,False,False,-1.790420,...,0,0,0,0,0,0,0,0,0,0
2,0,0,1,0.16,0.1364,0.47,0.3881,False,False,-2.180605,...,0,0,0,0,0,0,0,0,0,0
3,0,0,1,0.16,0.1364,0.47,0.2836,False,False,-2.061394,...,0,0,0,0,0,0,0,0,0,0
4,0,0,1,0.14,0.1061,0.50,0.3881,False,False,-2.177167,...,0,0,0,0,0,0,0,0,0,0


In [13]:
Y_train =wd_h['cnt'][wd_h['dteday'] < datetime.datetime(2012, 7, 5, 0, 0)]
Y_train.compute().head()

0     5
1     2
2     1
3     3
4    30
Name: cnt, dtype: int64

In [14]:
x_test = wd_h[wd_h['dteday'] > datetime.datetime(2012, 7, 5, 0, 0)].drop(['cnt','casual','registered','dteday','instant'],axis=1)
x_test.compute().head()

,yr,holiday,workingday,temp,atemp,hum,windspeed,isDaylight,isNoon,relative_temp,...,hr_16,hr_17,hr_18,hr_19,hr_20,hr_21,hr_22,hr_23,hr_2,hr_3
8976,1,0,1,0.78,0.7424,0.62,0.1343,False,False,-0.209613,...,0,0,0,0,0,0,0,0,0,0
8977,1,0,1,0.78,0.7273,0.55,0.2239,False,False,-0.212456,...,0,0,0,0,0,0,0,0,0,0
8978,1,0,1,0.76,0.7121,0.62,0.1343,False,False,-0.434529,...,0,0,0,0,0,0,0,0,1,0
8979,1,0,1,0.74,0.6970,0.70,0.1940,False,False,-0.657493,...,0,0,0,0,0,0,0,0,0,1
8980,1,0,1,0.74,0.6970,0.66,0.2239,False,False,-0.659466,...,0,0,0,0,0,0,0,0,0,0


In [15]:
y_test =wd_h['cnt'][wd_h['dteday'] > datetime.datetime(2012, 7, 5, 0, 0)]
y_test.compute().head()

8976    102
8977     29
8978     17
8979      7
8980      6
Name: cnt, dtype: int64

In [16]:
X_train = X_train#.values
Y_train = Y_train#.values

x_test = x_test#.values
y_test = y_test#.values

## Linear regression w/ GS for *Weekdays* for 2011 -> 2012Q3

Linear Regression using dask provides error "TypeError: unsupported operand type(s) for /: 'NoneType' and 'int'" with the same code that ran for baseline.

In [17]:
# from sklearn.metrics import r2_score

# #X_Train, Y_train = make_regression()
# lrd = LinearRegression() #Linear regression with dask_ml
# lrd.fit(X_train.values, Y_train.values)
# y_pred = lrd.predict(X_Test.values)
# r2_score(y_test.compute(), y_pred.compute())

### Linear Regression using SKLEARN & Dask Dataframes

In [20]:
from sklearn.linear_model import LinearRegression
from sklearn.externals import joblib

lm_parameters = {'fit_intercept':[True,False] ,'normalize':[True,False]}

lm = GridSearchCV(LinearRegression(),
                                 param_grid=lm_parameters,
                                 cv=tscv ,return_train_score=True)
with joblib.parallel_backend('dask'):
    lm.fit(X_train, Y_train)
lm.cv_results_
lm_predictions_w = lm.predict(x_test)
lm.score(x_test, y_test)

0.7742681424381046

### Random Forest using SKLEARN & Dask Dataframes

In [21]:
RF_parameters = {'n_estimators': [10, 30 ,100],
                                             'bootstrap': [True],
                                             'max_depth': [80, 100 ],
                                             'max_features': ['sqrt',16 ,32],
                                             'min_samples_leaf': [2,  5 , 8],
                                             'min_samples_split': [ 10 , 8 , 15],
                                            'random_state':[random_seed],
                                             "n_jobs": [-1],
                                            'criterion':['mse']}
rf = GridSearchCV(RandomForestRegressor(),
                                 param_grid= RF_parameters,
                                 cv=tscv)
with joblib.parallel_backend('dask'):
    rf.fit(X_train, Y_train)
rf.cv_results_
rf_predictions_w = rf.predict(x_test)
rf.score(x_test, y_test)


0.8691113616534034